<a href="https://colab.research.google.com/github/valerio-unifei/ECOP06/blob/main/ECOP06_06_Interface.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Interface

Disponibilidade de atributos públicos definidos (variáveis e métodos), não tem construtor e define o funcionamento de classes herdadas.

Não existe uma palavra reservada de Interface no Python, diferente de outras linguagem.

Toda a classe possui uma interface, inclusive classes abstratas.

In [ ]:
import abc

class FourWheelVehicle (abc.ABC):
  @abc.abstractmethod
  def SpeedUp( self ):
    pass

class Car(FourWheelVehicle) :
  def SpeedUp(self):
    print(" Running! ")

class TwoWheelVehicle (abc.ABC) :
  @abc.abstractmethod
  def SpeedUp(self):
    pass

class Bike(TwoWheelVehicle) :
  def SpeedUp(self) :
    print(" Running!.. ")

Determine as heranças das classes com o comando "isinstance".

In [ ]:
a = Bike ()
s = Car()

# Iteradores

Implementando os comandos:


```
def __iter__(self):
  pass

def __next__(self):
  pass
```



In [ ]:
class MyNumbers:
  def __iter__(self):
    self.a = 1
    return self

  def __next__(self):
    x = self.a
    self.a += 1
    return x

myobj = MyNumbers()
myiter = iter(myobj)

Utilizando:


```
next(<iterador>)
```



In [ ]:
next(myiter)

Utilizando a repetição:

In [ ]:
for x in myiter:
  print(x)

# Composição

Composição é um conceito que modela uma relação.

Ele permite criar tipos complexos combinando objetos de outros tipos.

As classes que contêm objetos de outras classes geralmente são chamadas de compostos, onde as classes usadas para criar tipos mais complexos são chamadas de componentes.

In [ ]:
class Component:
  def __init__(self):
    print('Component class object created...')

  def m1(self):
    print('Component class m1() method executed...')

class Composite:
	def __init__(self):
		self.obj1 = Component()
		print('Composite class object also created...')

	def m2(self):
		print('Composite class m2() method executed...')
		self.obj1.m1()

obj2 = Composite()
obj2.m2()

## Aplicação Real

Utilizando o serviço de API do INPE para obter previsão do tempo dos próximos dias.

Obtendo o ID da cidade no sistema do INPE em XML

In [ ]:
import requests
import urllib.parse
from xml.etree import ElementTree

city_url = 'http://servicos.cptec.inpe.br/XML/listaCidades?city=' + urllib.parse.quote(input())
response = requests.get(city_url)
response.content

Organizando XML para dicionário:

In [ ]:
tree = ElementTree.fromstring(response.content)
cidades=[]
for c in tree:
  ct={}
  for a in c:
    ct[a.tag]=a.text
  cidades.append(ct)
cidades

Obtendo XML com dados das previsões:

In [ ]:
cidade_id = int(cidades[0]['id'])
prev_url = f'http://servicos.cptec.inpe.br/XML/cidade/{cidade_id}/previsao.xml'
response = requests.get(prev_url)
dias = ElementTree.fromstring(response.content)
for d in dias:
  for n in d:
    print(n.tag,':',n.text)

**Proposta**: construa um conjunto de classes para armazenar os dados de previsão para um conjunto de cidades fornecido.

In [ ]:
import requests
import urllib.parse
from xml.etree import ElementTree

def ExtractContent(content):
  arvore = ElementTree.fromstring(content)
  itens = []
  for ramo in arvore:
    item = {}
    for folha in ramo:
      item[folha.tag] = folha.text
    if len(item.keys()) > 0:
      itens.append(item)
  return itens

# classe de previsao
class Previsao:
  def __init__(self, cidade_id):
    self.cidade_id = cidade_id
    prev_url = f'http://servicos.cptec.inpe.br/XML/cidade/{cidade_id}/previsao.xml'
    response = requests.get(prev_url)
    self.dias = ExtractContent(response.content)
  def __repr__(self):
    saida = ''
    for dia in self.dias:
      saida+= '{} [máxima: {} mínima: {}] {} \n'.format(
          dia['dia'],dia['maxima'],dia['minima'],dia['tempo'],
      )
    return saida

# criando a classe que busca a cidade
class Cidade:
  def __init__(self, cidade_nome:str):
    self.cidade_nome = cidade_nome
    self.cidade_inpe = []
    self.previsao = None
  def carregar(self):
    self.city_url = 'http://servicos.cptec.inpe.br/XML/listaCidades?city=' + urllib.parse.quote(self.cidade_nome)
    response = requests.get(self.city_url)
    # pegando a 1a cidade
    self.cidades_inpe = ExtractContent(response.content)
    if len(self.cidades_inpe) > 0:
      self.previsao = Previsao(self.cidades_inpe[0]['id'])
    return self
  def __repr__(self):
    if len(self.cidades_inpe) > 0:
      saida = '{}-{}'.format(self.cidades_inpe[0]['nome'],self.cidades_inpe[0]['uf'])
      saida += '\n'
      saida +=  self.previsao.__repr__()
      return saida
    else:
      return 'Cidade não encontrada'

ci = Cidade('Sorocaba')
ci.carregar()